In [1]:
import numpy as np
import pandas as pd

In [2]:
def make_guess2(guess, correct_chars, words):

    print('Correct letters:')
    print(correct_chars)
    
    incorrect_chars = [c for c in guess if c not in correct_chars]
    
    print('Incorrect letters:')
    print(incorrect_chars)
        
    # only words with no incorrect letters are kept
    if guess in words:
        words.remove(guess)
    else:
        print('Guessed a word not on remaining list..clever')
    no_incorrect = []
    for w in words:
        chars = [c for c in w]
        if (len([c for c in chars if c in incorrect_chars])==0):
                no_incorrect.append(w)

    remaining_words = []    
    if len(correct_chars)>0:
        for w in no_incorrect:
            chars = [c for c in w]
            if len(set([c for c in chars if c in correct_chars]))==len(correct_chars):
                remaining_words.append(w)
    else:
        remaining_words = no_incorrect

    print("Remaining words ({}) ".format(len(remaining_words)))     
    #print(remaining_words) 
    if len(remaining_words)<=3:
        print(remaining_words) 
        if len(remaining_words)==1:
            print('Congratulations...eliminated down to one!')


    return remaining_words

In [3]:
def split_by(words, char):
    
    split_with = set([w for w in words if char in w])
    split_without = set([w for w in words if char not in w])
    
    return split_with, split_without


In [4]:
def evaluate_guess(guess, target, position=None):
    
    correct_chars = [c for c in guess if c in target]
    
    if position == True:
        correct_chars_dict = dict()
        for c in correct_chars:
            correct_chars = [c for c in guess if c in target]
            for c in correct_chars:
                out = []
                idx_target = [i.start() for i in re.finditer(c, target)]
                idx_guess  = [i.start() for i in re.finditer(c, guess)]
                out = (out + 
                       [i+1 for i in idx_guess if i in idx_target] + 
                       [-(i+1) for i in idx_guess if i not in idx_target])
                correct_chars_dict[c] = list(set(out))
                
        correct_chars = correct_chars_dict
    else:
        correct_chars = list(set(correct_chars))
    
    
    return correct_chars

In [5]:
def make_guess(guess, target, words):

    correct_chars = set([c for c in guess if c in target])
    incorrect_chars = set([c for c in guess if c not in target])
    
    print('Correct letters:')
    print(correct_chars)
    print('Incorrect letters:')
    print(incorrect_chars)
    
    if guess==target:
        print('Congratulations...you guessed it!')
        
    else:
    
        # only words with no incorrect letters are kept
        if guess in words:
            words.remove(guess)
        else:
            print('Guessed a word not on remaining list..clever')
        no_incorrect = []
        for w in words:
            chars = [c for c in w]
            if (len([c for c in chars if c in incorrect_chars])==0):
                    no_incorrect.append(w)

        remaining_words = []    
        if len(correct_chars)>0:
            for w in no_incorrect:
                chars = [c for c in w]
                if len(set([c for c in chars if c in correct_chars]))==len(correct_chars):
                    remaining_words.append(w)
        else:
            remaining_words = no_incorrect

        print("Remaining words ({}): ".format(len(remaining_words)))     
        #print(remaining_words) 
        if len(remaining_words)<=3:
            print(remaining_words) 
            if len(remaining_words)==1:
                print('Congratulations...eliminated down to one!')

     
        return remaining_words
    

In [6]:
# def enter_result(guess, correct_chars, incorrect_chars, words):
    
#     print('Correct letters:')
#     print(correct_chars)
#     print('Incorrect letters:')
#     print(incorrect_chars)
    
#     # only words with no incorrect letters are kept
#     no_incorrect = []
#     for w in words:
#         chars = [c for c in w]
#         if (len([c for c in chars if c in incorrect_chars])==0):
#                 no_incorrect.append(w)

#     remaining_words = []    
#     if len(correct_chars)>0:
#         for w in no_incorrect:
#             chars = [c for c in w]
#             if len(set([c for c in chars if c in correct_chars]))==len(correct_chars):
#                 remaining_words.append(w)
#     else:
#         remaining_words = no_incorrect
    
#     print("Remaining words ({}): ".format(len(remaining_words)))     
#     #print(remaining_words)  

#     if len(remaining_words)==1:
#         print('Congratulations...you guessed it!')
    
#     if len(remaining_words)==1:
#         print('Congratulations...eliminated down to one!')
#         print(remaining_words) 
     
#     return remaining_words

In [7]:
def char_entropy(words_remaining, words_all):

    n_words = len(words_remaining)
    unique_chars = set([c for c in ''.join(words_all)])

    entropy = dict.fromkeys(unique_chars, 0)
    
    # get freq of letters in each word
    for char in unique_chars:
        n = 0
        for word in words_remaining:
            if char in word:
                n += 1

        if (n_words-n != 0) & (n != 0):
            entropy[char] = (-(n/n_words)*np.log2(n/n_words) 
                             - ((n_words-n)/n_words)*np.log2((n_words-n)/n_words))
        else:
            entropy[char] = 0

    return entropy


In [8]:
def get_word_entropy(words_remaining, words, return_best=None, show=None):
    # currently the information gain does not account for character position!
    
    word_entropy = dict.fromkeys(words, 0)
    char_ent = char_entropy(words_remaining, words)

    for w in words:
        entropy = 0
        used_chars = []
        for c in w:
             #if (c not in common_chars) & (c not in used_chars):
            if c not in used_chars:
                entropy = entropy + char_ent[c]
                used_chars.append(c)
        word_entropy[w] = entropy/len(w)
    
    best_choices = sorted(word_entropy, key=word_entropy.get, reverse=True)[:10]
    
    if word_entropy[best_choices[0]] == 0:
        print('Best entropy is 0...are all remaining words anagrams?')
    
    if show:
        print('Here are the best guesses...')
        for i in best_choices:
            print([i, word_entropy[i]])
            
    if return_best:
        return best_choices

In [9]:
def make_guess3(guess, correct, remaining_words):

    print('Correct letters:')
    print(correct)
    
    incorrect = [c for c in guess if c not in correct]
    
    print('Incorrect letters:')
    print(incorrect)

    incorrect_words = []
    for word in remaining_words:
        word_correct = True
        while(word_correct == True): 
            # correct characters (either in place or not)
            for char in correct:
                for i in correct[char]:
                    if i > 0:
                        idx = i-1
                        if word[idx] == char:
                            continue
                        else:
                            word_correct = False
                            incorrect_words.append(word)
                            break
                    else:
                        idx = abs(i)-1
                        if (word[idx] != char) & (char in word):
                            continue
                        else:
                            word_correct = False
                            incorrect_words.append(word)
                            break
            for char in incorrect:
                if (char in word):
                    incorrect_words.append(word)
                    break    
            break   

    remaining_words = [w for w in remaining_words if w not in incorrect_words]

    print("Remaining words ({}) ".format(len(remaining_words)))     
    if len(remaining_words)<=5:
        print(remaining_words) 
        if len(remaining_words)==1:
            print('Congratulations...eliminated down to one!')


    return remaining_words

In [10]:
with open('valid_solutions.csv') as f:
    lines = f.readlines()

wordle_words = [w[:5] for w in lines]

In [11]:
remaining_words = make_guess3('alert', {'l':[-2], 'e':[-3], 'r':[-4]}, wordle_words)

Correct letters:
{'l': [-2], 'e': [-3], 'r': [-4]}
Incorrect letters:
['a', 't']
Remaining words (20) 


In [12]:
get_word_entropy(remaining_words, wordle_words, show=True)

Here are the best guesses...
['opium', 0.6548323013965144]
['pubic', 0.6266633591710906]
['pivot', 0.5975529099733232]
['ficus', 0.5760768589114743]
['optic', 0.546966409713707]
['topic', 0.546966409713707]
['voice', 0.546966409713707]
['curio', 0.5469664097137069]
['music', 0.5395571316168095]
['spicy', 0.5395571316168094]


In [13]:
remaining_words = make_guess3('opium', {'o':[-1]}, remaining_words)

Correct letters:
{'o': [-1]}
Incorrect letters:
['p', 'i', 'u', 'm']
Remaining words (3) 
['loser', 'lover', 'lower']


In [14]:
get_word_entropy(remaining_words, wordle_words, show=True)

Here are the best guesses...
['askew', 0.36731833362179583]
['awash', 0.36731833362179583]
['salve', 0.36731833362179583]
['salvo', 0.36731833362179583]
['savor', 0.36731833362179583]
['savoy', 0.36731833362179583]
['savvy', 0.36731833362179583]
['scowl', 0.36731833362179583]
['screw', 0.36731833362179583]
['serve', 0.36731833362179583]


In [15]:
remaining_words = make_guess3('askew', {'s':[-2],'e':[4]}, remaining_words)

Correct letters:
{'s': [-2], 'e': [4]}
Incorrect letters:
['a', 'k', 'w']
Remaining words (1) 
['loser']
Congratulations...eliminated down to one!


In [16]:
get_word_entropy(remaining_words, wordle_words, show=True)

Best entropy is 0...are all remaining words anagrams?
Here are the best guesses...
['word\n', 0.0]
['aback', 0.0]
['abase', 0.0]
['abate', 0.0]
['abbey', 0.0]
['abbot', 0.0]
['abhor', 0.0]
['abide', 0.0]
['abled', 0.0]
['abode', 0.0]


In [17]:
remaining_words = make_guess3('above', {'o':[-3], 'e':[-5]}, remaining_words)

Correct letters:
{'o': [-3], 'e': [-5]}
Incorrect letters:
['a', 'b', 'v']
Remaining words (1) 
['loser']
Congratulations...eliminated down to one!


# Simulation

In [18]:
with open('valid_solutions.csv') as f:
    lines = f.readlines()

In [19]:
wordle_words = [w[:5] for w in lines]
wordle_words.remove('word\n')

target_words = wordle_words

results = dict()

for target in target_words:
    
    results[target] = dict()
    guesses = []
    #wordle_words = [w[:5] for w in lines]
    remaining_words = [w[:5] for w in lines]
    finished_game = False

    print('<<<<< New game: ' + target + ' >>>>>')

    while ~finished_game:
        for i in range(10):
            if len(remaining_words) == 2:
                next_guess = remaining_words[0]
                
            else:
                next_guess = get_word_entropy(remaining_words, wordle_words, return_best=True)[0]
                
            if next_guess != target:
                guesses.append(next_guess)
                print('Guessing: ' + next_guess)
                correct_chars = evaluate_guess(next_guess, target, position=True)
                remaining_words = make_guess3(next_guess, correct_chars, remaining_words)

                if len(remaining_words) <= 1:
                    finished_game = True
                    if len(remaining_words) == 0:
                        print('<<<<< Ran out of guesses..anagrams? >>>>>>')
                        results[target]['n'] = i+2
                        results[target]['guesses'] = guesses
                    elif remaining_words[0] == target:
                        print('<<<<< Guess: ' + remaining_words[0] + ', game finished >>>>>>')
                        results[target]['n'] = i+2
                        guesses.append(remaining_words[0])
                        results[target]['guesses'] = guesses

                    break
            else:
                finished_game = True
                print('<<<<< Guess: ' + next_guess + ', game finished >>>>>>')
                results[target]['n'] = i+2
                guesses.append(next_guess)
                results[target]['guesses'] = guesses
                break
                


        break
        
    
results = pd.DataFrame(results).T.reset_index().rename(columns={'index': 'target'})    

<<<<< New game: aback >>>>>
Guessing: alert


NameError: name 're' is not defined

In [ ]:
results['n'].plot.hist()

In [ ]:
results[results['n'] > 6]

In [ ]:
results[results['target'] == 'blown']

# Class definition

In [3]:
class PlayWorldle(object):
    
    def __init__(self, all_words):
        print('<<< Welcome to a new Worldle! >>>')
        self.all_words = all_words           # all words in dictionary
        self.remaining_words = all_words     # remaining words
        self.guesses = []                    # guesses

    def enter_guess(self, guess, correct):
        
        self.guesses = self.guesses + [guess]

        print('Correct letters:')
        print(correct)

        incorrect = [c for c in guess if c not in correct]

        print('Incorrect letters:')
        print(incorrect)
        
        remaining_words = self.remaining_words

        incorrect_words = []
        for word in remaining_words:
            word_correct = True
            while(word_correct == True): 
                for char in correct:
                    for i in correct[char]:
                        if i > 0:
                            idx = i-1
                            if word[idx] == char:
                                continue
                            else:
                                word_correct = False
                                incorrect_words.append(word)
                                break
                        else:
                            idx = abs(i)-1
                            if (word[idx] != char) & (char in word):
                                continue
                            else:
                                word_correct = False
                                incorrect_words.append(word)
                                break
                for char in incorrect:
                    if (char in word):
                        incorrect_words.append(word)
                        break    
                break   

        remaining_words = [w for w in remaining_words if w not in incorrect_words]

        print("Remaining words ({}) ".format(len(remaining_words)))     
        if len(remaining_words)<=5:
            print(remaining_words) 
            if len(remaining_words)==1:
                print('Congratulations...eliminated down to one!')

        self.remaining_words = remaining_words
        

    def gen_char_entropy(self):

        words_remaining = self.remaining_words
        words_all = self.all_words
        n_words = len(words_remaining)
        unique_chars = set([c for c in ''.join(words_all)])

        entropy = dict.fromkeys(unique_chars, 0)

        # get freq of letters in each word
        for char in unique_chars:
            n = 0
            for word in words_remaining:
                if char in word:
                    n += 1
            if (n_words-n != 0) & (n != 0):
                entropy[char] = (-(n/n_words)*np.log2(n/n_words) 
                                 - ((n_words-n)/n_words)*np.log2((n_words-n)/n_words))
            else:
                entropy[char] = 0

        return entropy 

        
    def gen_word_entropy(self, return_best=None, show=None):
        
        words_remaining = self.remaining_words
        words = self.all_words

        word_entropy = dict.fromkeys(words, 0)
        char_ent = self.gen_char_entropy()

        for w in words:
            entropy = 0
            used_chars = []
            for c in w:
                if c not in used_chars:
                    entropy = entropy + char_ent[c]
                    used_chars.append(c)
            word_entropy[w] = entropy/len(w)

        best_choices = sorted(word_entropy, key=word_entropy.get, reverse=True)[:10]

        if word_entropy[best_choices[0]] == 0:
            print('Best entropy is 0...are all remaining words anagrams?')

        if show:
            print('Here are the best guesses...')
            for i in best_choices:
                print([i, word_entropy[i]])

        if return_best:
            return best_choices
        
        
       

In [15]:
with open('valid_solutions.csv') as f:
    lines = f.readlines()

wordle_words = [w[:5] for w in lines]

obj = PlayWorldle(wordle_words)

<<< Welcome to a new Worldle! >>>


In [16]:
obj.enter_guess('alert', {'a':[1], 'l':[2], 'e':[-3]})

Correct letters:
{'a': [1], 'l': [2], 'e': [-3]}
Incorrect letters:
['r', 't']
Remaining words (6) 


In [17]:
obj.gen_word_entropy(show=True)

Here are the best guesses...
['vying', 0.7736726197999104]
['bingo', 0.6436681354702396]
['dingo', 0.6436681354702396]
['dingy', 0.6436681354702396]
['doing', 0.6436681354702396]
['dying', 0.6436681354702396]
['eking', 0.6436681354702396]
['eying', 0.6436681354702396]
['given', 0.6436681354702396]
['going', 0.6436681354702396]


In [18]:
obj.enter_guess('vying', {'i':[3]})

Correct letters:
{'i': [3]}
Incorrect letters:
['v', 'y', 'n', 'g']
Remaining words (1) 
['alike']
Congratulations...eliminated down to one!


In [13]:
obj.gen_word_entropy(show=True)

Here are the best guesses...
['acrid', 0.36731833362179583]
['caddy', 0.36731833362179583]
['cadet', 0.36731833362179583]
['candy', 0.36731833362179583]
['cedar', 0.36731833362179583]
['chard', 0.36731833362179583]
['chide', 0.36731833362179583]
['child', 0.36731833362179583]
['chord', 0.36731833362179583]
['cider', 0.36731833362179583]


In [ ]:
obj.enter_guess('caddy', {'h':[-2]})

In [ ]:
obj.gen_word_entropy(show=True)

In [ ]:
obj.enter_guess('vowel', {'v':[-1], 'e':[4]})